In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tensorflow.examples.tutorials.mnist import input_data


%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
location = 'H:/data/'
mnist = input_data.read_data_sets(location,one_hot=True)
print('\nDownload finished!')


#设定超参数
learning_rate = 0.01
training_iters = 5000
batch_size = 64
display_step = 10


n_inputs = 28  #列数目
n_steps = 28   #行数目
n_hidden_units = 128  #隐藏层神经元的个数是128个
n_classes = 10  #总共有10类


#Graph 输入
x = tf.placeholder(dtype=tf.float32,shape=[None,n_steps,n_inputs])
y = tf.placeholder(dtype=tf.float32,shape=[None,n_classes])

Extracting H:/data/train-images-idx3-ubyte.gz
Extracting H:/data/train-labels-idx1-ubyte.gz
Extracting H:/data/t10k-images-idx3-ubyte.gz
Extracting H:/data/t10k-labels-idx1-ubyte.gz

Download finished!


In [3]:
Weights = {
    'in':  tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}

Biases  = {
    'in':  tf.Variable(tf.random_normal([n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

实验证明一下两种方法都可以进行多层LSTM的定义
==
方法一：

------------------------------------------
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell]*2) # 2层LSTM



方法二：

------------------------------------------
def get_a_cell():

       return tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell() for _ in range(2)]) # 2层LSTM

In [4]:
def get_a_cell():
    return tf.nn.rnn_cell.BasicLSTMCell(num_units=128)

def RNN(X,weight,biase):
    #hidden layer for input
    #X --> 128 * 28行 * 28列 
    X = tf.reshape(X,[-1,n_inputs])
    X_in = tf.matmul(X,weight['in']) + biase['in']
    X_in = tf.reshape(X_in,[-1,n_steps,n_hidden_units])
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)
#     multi_cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell]*2)
    multi_cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell() for _ in range(2)]) # 2层LSTM
    #cell

    init_state = multi_cell.zero_state(batch_size,dtype=tf.float32)
    output,states = tf.nn.dynamic_rnn(multi_cell,X_in,initial_state=init_state,time_major=False)
    print(np.shape(states))
    #hidden layer for output
    result = tf.matmul(states[1][1],Weights['out'])+biase['out']
    return result


pred = RNN(x,Weights,Biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred))

train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)


correct_pred = tf.equal(tf.arg_max(pred,1),tf.arg_max(y,1))
accuracy_caculate = tf.reduce_mean(tf.cast(correct_pred,tf.float32))


init = tf.global_variables_initializer()


(2, 2)


In [5]:
with tf.Session() as sess:
    sess.run(init)
    steps = 0
    for steps  in range(training_iters):
        batch = mnist.train.next_batch(batch_size)
        batch_x = batch[0].reshape([-1,n_steps,n_inputs])
        batch_y = batch[1]
        
        sess.run(train_op,feed_dict = {x:batch_x, y:batch[1]})
        
        if steps%200 ==0:
            print('iter NO: ',steps,' 准确率是：',sess.run(accuracy_caculate,feed_dict = { x:batch_x, y:batch_y}),\
                 '   损失： ',sess.run(cost,feed_dict = { x:batch_x, y:batch_y}))
        steps+=1

iter NO:  0  准确率是： 0.203125    损失：  10.841012
iter NO:  200  准确率是： 0.671875    损失：  0.7730325
iter NO:  400  准确率是： 0.96875    损失：  0.20593289
iter NO:  600  准确率是： 0.859375    损失：  0.30317616
iter NO:  800  准确率是： 0.9375    损失：  0.36036387
iter NO:  1000  准确率是： 1.0    损失：  0.058910318
iter NO:  1200  准确率是： 0.96875    损失：  0.09840456
iter NO:  1400  准确率是： 0.984375    损失：  0.096673444
iter NO:  1600  准确率是： 0.96875    损失：  0.13619779
iter NO:  1800  准确率是： 0.984375    损失：  0.09508893
iter NO:  2000  准确率是： 0.953125    损失：  0.10007714
iter NO:  2200  准确率是： 0.984375    损失：  0.05720318
iter NO:  2400  准确率是： 0.984375    损失：  0.06994754
iter NO:  2600  准确率是： 0.96875    损失：  0.14140889
iter NO:  2800  准确率是： 0.953125    损失：  0.1021637
iter NO:  3000  准确率是： 0.9375    损失：  0.12517326
iter NO:  3200  准确率是： 0.984375    损失：  0.05231764
iter NO:  3400  准确率是： 0.96875    损失：  0.122942775
iter NO:  3600  准确率是： 1.0    损失：  0.026326928
iter NO:  3800  准确率是： 0.984375    损失：  0.07332572
iter NO:  4000  准确率是： 0.9